In [1]:
import wrds
import pandas as pd
import datetime
import numpy as np
import pickle
from time import time

In [2]:
db = wrds.Connection()

Enter your WRDS username [liqiran]:ansonccy
Enter your password:········
WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.
Loading library list...
Done


In [3]:
q = ("select a.gvkey,a.latest,b.cshoq,b.prccq,b.mkvaltq,b.cshoq*b.prccq as market_cap,b.curcdq "
     "from "
        "(select gvkey,max(datadate) as latest "
         "from "
         "compm.fundq where datadate > '2017-01-01' " 
         "group by gvkey) a inner join "
             "(select gvkey,datadate,mkvaltq,cshoq,prccq,curcdq "
                "from compm.fundq where cshoq>0 and prccq>0 and curcdq='USD') b "
    "on a.gvkey = b.gvkey and a.latest=b.datadate "
     "order by market_cap desc "
    "limit 100")

In [4]:
a = db.raw_sql(q)
top_2000_eq_gvkey = a['gvkey'].values.tolist()
top_2000_eq_gvkey = tuple(["'%s'"%str(i) for i in top_2000_eq_gvkey])
top_2000_eq_gvkey = ",".join(top_2000_eq_gvkey)

In [5]:
q2 = ("select datadate,gvkey,tic,saleq,cogsq,xsgaq,oiadpq,niq,"
      "cheq, rectq, invtq, acoq, ppentq, aoq, dlcq, apq, txpq, lcoq, ltq, dlttq,cshoq "
    "from compm.fundq "
     "where gvkey in (%s) ")%top_2000_eq_gvkey
c = db.raw_sql(q2)
c.shape

(13979, 21)

In [6]:
blnc_sheet_list = ['cheq','rectq','invtq','acoq','ppentq','aoq',
                                'dlcq','apq','txpq','lcoq','ltq','dlttq','cshoq']
mqr_list = [x + '_mqr' for x in blnc_sheet_list]

In [7]:
income_list = ['saleq','cogsq','xsgaq','oiadpq','niq']
x = c[['gvkey','datadate'] + income_list + blnc_sheet_list]
x = x[x['gvkey'] == '006066']
x.sort_values('datadate').head()

,gvkey,datadate,saleq,cogsq,xsgaq,oiadpq,niq,cheq,rectq,invtq,acoq,ppentq,aoq,dlcq,apq,txpq,lcoq,ltq,dlttq,cshoq
4647,006066,1962-03-31,453.2,NaN,NaN,NaN,56.264,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4648,006066,1962-06-30,478.5,NaN,NaN,NaN,60.045,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4649,006066,1962-09-30,467.7,NaN,NaN,NaN,59.213,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4650,006066,1962-12-31,525.8,NaN,NaN,461.94,65.865,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4651,006066,1963-03-31,666.4,NaN,NaN,NaN,78.593,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
t1 = time()
%run wrds_data_processing.py
print time()-t1

IOError: [Errno 2] No such file or directory: 'y.pkl'

0.0225591659546


In [9]:
qq = ("select gvkey,datadate,prccm "
     "from compm.secm "
     "where gvkey='160329' ")
d = db.raw_sql(qq).sort_values('datadate')
d = d.set_index('datadate')

In [10]:
d.loc[datetime.date(2014,3,31)]

,gvkey,prccm
datadate,,
2014-03-31,160329,1114.51
2014-03-31,160329,556.97


In [11]:
qq = ("select * "
     "from compm.secm "
     "where gvkey='001690' ")
d = db.raw_sql(qq).sort_values('datadate')
d = d.set_index('datadate')

In [12]:
d.loc[datetime.date(2014,3,31)]

gvkey             001690
iid                   01
tic                 AAPL
cusip          037833100
conm           APPLE INC
ajexm                  7
ajpm                   7
isalrt              None
primiss                P
spgim           45202030
spiim               None
spmim                 10
cheqvm              None
curcddvm             USD
dvpspm               NaN
dvpsxm               NaN
dvrate              12.2
csfsm               None
cshtrm       1.78201e+08
curcdm               USD
navm                None
prccm             536.74
prchm                549
prclm             522.81
trfm             1.13951
trt1m            1.99529
rawpm                NaN
rawxm                NaN
sph100              None
sphcusip            None
sphiid               NaN
sphmid               NaN
sphname             None
sphsec               NaN
sphtic              None
sphvg               None
cshoq            861.745
adrrm               None
cmth                   3
cshom        8.92554e+08


In [13]:
d.to_pickle("prccm.pkl")

In [14]:
q5 = ("select gvkey,datadate,split "
     "from compm.sec_split "
     "where gvkey='160329' ")
e = db.raw_sql(q5).sort_values('datadate')

In [15]:
e= pd.read_pickle('prccm.pkl')

In [16]:
from time import time

In [17]:
t1 = time()
%run build_data.py
print time()-t1

Enter your WRDS username [liqiran]:ansonccy
Enter your password:········
WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.
Loading library list...
Done
Shape of raw dataframe: 194396,21


2000
3687.57893395
3687.58392215


In [18]:
y = pd.read_pickle('y.pkl')

IOError: [Errno 2] No such file or directory: 'y.pkl'

In [ ]:
y.iloc[-10::]

In [ ]:
21771+19226+20238+18685